In [0]:
from pyspark.sql.functions import col, isnull, when, udf
from pyspark.sql.types import TimestampType
from pyspark.sql.types import StringType
from pyspark.sql.functions import round
import pandas as pd
from datetime import datetime

spark.conf.set(
  "fs.azure.account.key.doanquanglam.blob.core.windows.net",
  "6b+5X1whqm5IG8eQDg/0AAIU5tiu3aEXgmqJrBheRTjmmnS80mCFb7yeirJXZf16oYfV97wdwZJx+AStb2G5RQ=="
)

In [0]:
dbutils.widgets.text("blob_path", "")
blob_path = dbutils.widgets.get("blob_path")


In [0]:
#  xử lý chuỗi để spark có thể đọc được file 
# Chuyển đổi sang định dạng wasbs
from urllib.parse import urlparse

parsed_url = urlparse(blob_path)
container = parsed_url.path.split('/')[1]
path_in_container = '/'.join(parsed_url.path.split('/')[2:])
account = parsed_url.netloc.split('.')[0]
wasbs_path = f"wasbs://{container}@{account}.blob.core.windows.net/{path_in_container}"

js = spark.read.format("json").load(wasbs_path)


✅ wasbs path: wasbs://rawdata@doanquanglam.blob.core.windows.net/234/test_data.json


In [0]:
df = (
    js
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [0]:
# chuyển đổi 'time' và 'updated' từ timestamp sang Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)


today_str = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

# output_path = f"wasbs://silver@doanquanglam.blob.core.windows.net/{today_str}/"
output_path = f"wasbs://silver@doanquanglam.blob.core.windows.net/"

df.write.mode('append').parquet(output_path)

# lớp vàng 

# thay giá trị null của latitude và longtude thành 0 nếu nó là null
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
)

In [0]:
df = df.filter(df.id.isNotNull())

In [0]:
df = df.withColumn('sig_class', when(col("sig") < 100, "Low").when((col("sig") >= 100) & (col("sig") < 500), "Moderate").otherwise("High"))

In [0]:
df = df.filter(df.magType.isNotNull())

In [0]:
# làm tròn giá trị 
df = (
    df
    .withColumn("longitude", round(col("longitude"), 5))
    .withColumn("latitude", round(col("latitude"), 5))
    .withColumn("longitude", round(col("longitude"), 5))
    .withColumn("elevation", round(col("elevation"), 5))
    .withColumn("mag", round(col("mag"), 2))
)

In [0]:


# Tạo đường dẫn đầy đủ
# output_path = f"wasbs://gold@doanquanglam.blob.core.windows.net/{today_str}/"
output_path = f"wasbs://gold@doanquanglam.blob.core.windows.net/"

# Ghi dữ liệu Parquet
df.write.mode("overwrite").option("header", True).parquet(output_path)

